In [ ]:
# pip installs

!pip install -q datasets requests torch peft bitsandbytes transformers trl accelerate sentencepiece wandb matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# imports
# With much thanks to Islam S. for identifying that there was a missing import!

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata, drive
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig, PeftModel, PeftConfig
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/lora-checkpoints"

In [ ]:
!ls "/content/drive/MyDrive/"

 008.jpg
'231129 GM Agenda (1).gdoc'
'231129 GM Agenda (2).gdoc'
'231129 GM Agenda (3).gdoc'
'231129 GM Agenda.gdoc'
'231129 GM Agenda.pdf'
'Accenture Application Form - Singapore.xlsm'
 ACE_Workbook_v2.0（副本）.gslides
 Algo-DataStructure.gdoc
'AWS Certified AI Practitioner (AIF-C01).gdoc'
 AWS-Certified-ML-Engineer-Associate-Slides.pdf
'AWS Certified Solutions Architect Associate SAA-C03.gdoc'
'AY 2017 Sem 1 E341_W04_WS.gdoc'
'AY 2017 Sem 1 E344_W07_LAB (1).doc.gdoc'
'AY 2017 Sem 1 E344_W07_LAB.doc'
'AY 2017 Sem 1 E344_W07_LAB.doc.gdoc'
'AY 2017 Sem 1 E344_W08_LAB.gdoc'
'AY 2017 Sem 1 E344_W08_WS.doc'
'AY 2017 Sem 1 E344_W08_WS.doc.gdoc'
'AY2017 Sem 1 E345_W12_WS.gdoc'
'AY 2017 Sem 1 E348_W03_WS.gdoc'
 codeStructure.drawio
 CodingAssignment_Yifei.zip
'Colab Notebooks'
 Computer_Organization_and_Design_5th_Edi.pdf
'Copy of cloud_platforms_information_matrix_v4.xlsx'
'CZ4067_Experience_Report(KopiO).gdoc'
 CZ4067_Experience_Report_MOCK_JIN_WEI_1721242F.gdoc
 DataFiles
'Develop Guide.gdoc'

In [ ]:
drive.flush_and_unmount()

In [ ]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "pricer"
HF_USER = "AI-weng" # your HF name here!

# Data

DATASET_NAME = f"{HF_USER}/pricer-data"
# Or just use the one I've uploaded
# DATASET_NAME = "ed-donner/pricer-data"
MAX_SEQUENCE_LENGTH = 182

# Run name for saving the model in the hub

RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

# Resume checkpoint

# RUN_NAME = "2025-01-02_09.12.32"
# PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
# HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"


# Hyperparameters for QLoRA

LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
LORA_DROPOUT = 0.1
QUANT_4_BIT = True

# Hyperparameters for Training

EPOCHS = 3
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE = 'cosine'
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"

# Admin config

STEPS = 50
SAVE_STEPS = 5000
LOG_TO_WANDB = True

%matplotlib inline

In [ ]:
HUB_MODEL_NAME

'AI-weng/pricer-2025-01-03_11.38.40'

In [ ]:
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# Log in to Weights & Biases
wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: johnewong95 (yfweng). Use `wandb login --relogin` to force relogin


In [ ]:
dataset = load_dataset(DATASET_NAME)
test_dataset = dataset['test']

In [ ]:
dataset_train_val = dataset["train"].train_test_split(test_size=0.0125, seed=42)
train_dataset = dataset_train_val["train"]   # ~395k
val_dataset   = dataset_train_val["test"]

In [ ]:
train_dataset

Dataset({
    features: ['text', 'price'],
    num_rows: 395000
})

In [ ]:
val_dataset

Dataset({
    features: ['text', 'price'],
    num_rows: 5000
})

In [ ]:
test_dataset[0]

{'text': "How much does this cost to the nearest dollar?\n\nOEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K\n\nPrice is $",
 'price': 374.41}

In [ ]:
if LOG_TO_WANDB:
  wandb.init(project=PROJECT_NAME, name=RUN_NAME)

In [ ]:
# pick the right quantization

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
!pip install -U bitsandbytes
# Restart session now

In [ ]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Memory footprint: 5591.5 MB


In [ ]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "Price is $"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
# First, specify the configuration parameters for LoRA

lora_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
    inference_mode=False
)

# Next, specify the general configuration parameters for training

train_parameters = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="steps",
    eval_steps=500,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim=OPTIMIZER,
    save_steps=SAVE_STEPS,
    save_total_limit=10,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=WARMUP_RATIO,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="every_save",
    push_to_hub=True,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True,
    overwrite_output_dir=False
)

In [ ]:
from peft import prepare_model_for_kbit_training
base_model1 = prepare_model_for_kbit_training(base_model)

In [ ]:
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=lora_parameters,
    tokenizer=tokenizer,
    args=train_parameters,
    data_collator=collator
)

<ipython-input-18-46cb98592198>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  fine_tuning = SFTTrainer(


Map:   0%|          | 0/395000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# Fine-tune!
fine_tuning.train()

# Push our fine-tuned model to Hugging Face
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

Step,Training Loss,Validation Loss
500,1.870500,1.871360
1000,1.859200,1.857773
1500,1.851400,1.847639
2000,1.836100,1.835689
2500,1.841600,1.846857
3000,1.829500,1.830567
3500,1.829600,1.834868
4000,1.842100,1.823870
4500,1.821700,1.830976
5000,1.829600,1.830137


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

In [ ]:
import os
from transformers.trainer_utils import get_last_checkpoint

checkpoint_dir = None

if os.path.isdir(OUTPUT_DIR):
    # get_last_checkpoint returns the path to the most recently saved checkpoint
    last_checkpoint = get_last_checkpoint(OUTPUT_DIR)
    if last_checkpoint is not None:
        checkpoint_dir = last_checkpoint
        print(f"Resuming from last checkpoint: {checkpoint_dir}")
    else:
        print("No checkpoint found in Drive. Starting from scratch.")
else:
    print("No output directory found. Starting from scratch.")

# Initialize your Trainer (SFTTrainer) again
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    peft_config=lora_config,
    tokenizer=tokenizer,
    args=train_parameters,
    data_collator=data_collator
)

# If checkpoint_dir is None, training will start from scratch
fine_tuning.train(resume_from_checkpoint=checkpoint_dir)